In [1]:
import sys
sys.path.append("../../")
import qsimcirq
import cirq
import numpy as np
import importlib
importlib.reload(qsimcirq)
import matplotlib.pyplot as plt

Test Hadamard and CNOT (Bell state preparation)

In [2]:
# Define qubits
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)
q2 = cirq.LineQubit(2)

block_gate2 = qsimcirq.BlockGate([
    cirq.CNOT(q1, q0),
    cirq.CNOT(q1, q2)
])

# Create a circuit and add the BlockOperation
circuit_block = cirq.Circuit(
    cirq.H(q1),
    block_gate2(q0,q1,q2),  # Add the BlockOperation directly

)
print("Circuit (block):")
print(circuit_block)


Circuit (block):
0: ───────BlockGate(CNOT(q(1), q(0)), CNOT(q(1), q(2)))───
          │
1: ───H───#2──────────────────────────────────────────────
          │
2: ───────#3──────────────────────────────────────────────


In [3]:
circuit_noblock = cirq.Circuit(
    cirq.H(q1),
    cirq.CNOT(q1, q0),
    cirq.CNOT(q1, q2)    
)
print("Circuit (no block):")
print(circuit_noblock)

Circuit (no block):
0: ───────X───────
          │
1: ───H───@───@───
              │
2: ───────────X───


In [4]:
options = {}

# 'k' indicates the qubits on one side of the cut.
options['k'] = [0,1]

# 'p' and 'r' control when values are assigned to cut indices.
options['p'] = 0
options['r'] = 1

# 'w' indicates the value pre-assigned to the cut.
# This should change for each execution.
options['w'] = 0

# Create the qsimh simulator with those options.
qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_0 = qsimh_simulator.compute_amplitudes(
    circuit_block, bitstrings=[0b000, 0b001, 0b010, 0b011, 0b100, 0b101, 0b110, 0b111])

  ╷ 1
╶─┼───
0 │ H
  │
  ╷ 0                                             1  2
╶─┼─────────────────────────────────────────────────────
0 │ BlockGate(CNOT(q(1), q(0)), CNOT(q(1), q(2)))─#2─#3
  │
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.H(cirq.LineQubit(1)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.HPowGate'>
other case
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (BlockGate([cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(0)), cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(2))]).on(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'qsimcirq.block_gate.BlockGate'>
case block
gate_kind_temp GateKind.kCX
time 1
qubits_temp [1, 0]
params {'exponent': 1.0, 'global_shift': 0.0}
gates_temp after create_gate appended [<qsimcirq.qsim_avx512.GateCirq object at 0x7f0abb8a6430>]
gate_kind_temp Ga

In [5]:
#no cut
qsim_simulator = qsimcirq.QSimSimulator()
results_noblock = qsim_simulator.compute_amplitudes(circuit_noblock, bitstrings=[0b000, 0b001, 0b010, 0b011, 0b100, 0b101, 0b110, 0b111])
print("qsim results:")
print(results_noblock)

  ╷ 1
╶─┼───
0 │ H
  │
  ╷ 0 1
╶─┼─────
0 │ X─@
  │
  ╷ 1 2
╶─┼─────
0 │ @─X
  │
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.H(cirq.LineQubit(1)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.HPowGate'>
other case
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(0)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.CXPowGate'>
other case
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(2)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.CXPowGate'>
other case
qsim_circuit [<qsimcirq.qsim_avx512.GateCirq object at 0x7f0aba8900b0>, <qsimcirq.qsim_avx512.GateCirq object at 0x7f0ae8969270>, <qsimcirq.qsim_avx512.GateCirq object at 0x7f0ae8c9faf0>]
qsim results:

In [6]:
results_0 = np.array(results_0)
results_noblock = np.array(results_noblock)
np.linalg.norm(results_0 - results_noblock)

8.940696716308594e-08

A more random block to test whether flipped gates work (if they are not symmetric under flipping the qubits and I had to add a case)

In [7]:
# Define qubits
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)
q2 = cirq.LineQubit(2)

theta1 = 0.24
theta2 = 1.34

block_gate2 = qsimcirq.BlockGate([
    cirq.CXPowGate(exponent=theta1/np.pi, global_shift=-0.5).on(q2,q0),
    cirq.CXPowGate(exponent=theta2/np.pi, global_shift=-0.5).on(q1,q0),
    cirq.PhasedISwapPowGate(phase_exponent=2.5, exponent=2.1, global_shift=0.2).on(q2,q0),
    cirq.givens(angle_rads=0.2).on(q1,q0),  
])

# Create a circuit and add the BlockOperation
circuit_block = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    block_gate2(q0,q1,q2),  # Add the BlockOperation directly

)

print("Circuit (block):")
print(circuit_block)


Circuit (block):
0: ───H───BlockGate(CNOT**0.07639437268410976(q(2), q(0)), CNOT**0.4265352474862795(q(1), q(0)), PhasedISWAP(exponent=2.1, global_shift=0.2)(q(2), q(0)), PhasedISWAP**0.12732395447351627(q(1), q(0)))───
          │
1: ───H───#2──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │
2: ───H───#3──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [8]:
circuit_noblock = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.CXPowGate(exponent=theta1/np.pi, global_shift=-0.5).on(q2,q0),
    cirq.CXPowGate(exponent=theta2/np.pi, global_shift=-0.5).on(q1,q0),
    cirq.PhasedISwapPowGate(phase_exponent=2.5, exponent=2.1, global_shift=0.2).on(q2,q0),
    cirq.givens(angle_rads=0.2).on(q1,q0), 
)
print("Circuit (no block):")
print(circuit_noblock)

Circuit (no block):
0: ───H───X^(1/13)───X^0.427───PhISwap(0.5)────────PhISwap(0.25)─────────
          │          │         │                   │
1: ───H───┼──────────@─────────┼───────────────────PhISwap(0.25)^0.127───
          │                    │
2: ───H───@────────────────────PhISwap(0.5)^-1.9─────────────────────────


In [9]:
options = {}

# 'k' indicates the qubits on one side of the cut.
options['k'] = [0]

# 'p' and 'r' control when values are assigned to cut indices.
options['p'] = 0
options['r'] = 1

# 'w' indicates the value pre-assigned to the cut.
# This should change for each execution.
options['w'] = 0

# Create the qsimh simulator with those options.
qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_0 = qsimh_simulator.compute_amplitudes(
    circuit_block, bitstrings=[0b000, 0b001, 0b010, 0b011, 0b100, 0b101, 0b110, 0b111])

  ╷ 0 1 2
╶─┼───────
0 │ H H H
  │
  ╷ 0                                                                                                                                                                                             1  2
╶─┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
0 │ BlockGate(CNOT**0.07639437268410976(q(2), q(0)), CNOT**0.4265352474862795(q(1), q(0)), PhasedISWAP(exponent=2.1, global_shift=0.2)(q(2), q(0)), PhasedISWAP**0.12732395447351627(q(1), q(0)))─#2─#3
  │
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.H(cirq.LineQubit(0)), cirq.H(cirq.LineQubit(1)), cirq.H(cirq.LineQubit(2)))
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.HPowGate'>
other case
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (BlockGate([cirq

In [10]:
#no cut
qsim_simulator = qsimcirq.QSimSimulator()
results_noblock = qsim_simulator.compute_amplitudes(circuit_noblock, bitstrings=[0b000, 0b001, 0b010, 0b011, 0b100, 0b101, 0b110, 0b111])
print("qsim results:")
print(results_noblock)

  ╷ 0 1 2
╶─┼───────
0 │ H H H
  │
  ╷ 0                     2
╶─┼─────────────────────────
0 │ X^0.07639437268410976─@
  │
  ╷ 0                    1
╶─┼────────────────────────
0 │ X^0.4265352474862795─@
  │
  ╷ 0            2
╶─┼────────────────────────────────
0 │ PhISwap(0.5)─PhISwap(0.5)^-1.9
  │
  ╷ 0             1
╶─┼─────────────────────────────────────────────────
0 │ PhISwap(0.25)─PhISwap(0.25)^0.12732395447351627
  │
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.H(cirq.LineQubit(0)), cirq.H(cirq.LineQubit(1)), cirq.H(cirq.LineQubit(2)))
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.HPowGate'>
other case
moment type <class 'cirq.circuits.moment.Moment'> moment.operations (cirq.CXPowGate(exponent=0.07639437268410976, global_shift=-0.5).on(cirq.LineQubit(2), cirq.LineQubit(0)),)
moment.op type <class 'cirq.ops.gate_operation.GateOperation'>
moment gate <class 'cirq.ops.common_gates.CXPowGate

In [11]:
results_0 = np.array(results_0)
results_noblock = np.array(results_noblock)
np.linalg.norm(results_0 - results_noblock)

3.3052374441651755e-07